# 학교 안전사고 상관분석_학교급별

- 코드 작성자 : 이유정
- 코드 작성일 : 2024-07-23

## 0. 라이브러리 및 데이터 임포트

In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
import scipy.stats as stats
import plotly.express as px

In [4]:
df = pd.read_csv('../../../school-accidents/code/dashboard/dashboard_data/학교안전사고데이터통합/학교안전사고데이터_5개년통합_월계절추가.csv')

df['사고발생일'] = pd.to_datetime(df['사고발생일'])
df['연도'] = df['사고발생일'].map(lambda x : x.year)

df_2019 = df[df['사고발생일'].between('2019-01-01', '2019-12-31')]
df_2020 = df[df['사고발생일'].between('2020-01-01', '2020-12-31')]
df_2021 = df[df['사고발생일'].between('2021-01-01', '2021-12-31')]
df_2022 = df[df['사고발생일'].between('2022-01-01', '2022-12-31')]
df_2023 = df[df['사고발생일'].between('2023-01-01', '2023-12-31')]

##### 1-3-0. 데이터 일부 추출

In [5]:
# 기본 정보 확인
print(df.columns)
print(df.학교급.unique())
df.groupby('학교급').count()['구분']

Index(['구분', '학교급', '지역', '교육청', '설립유형', '사고자구분', '사고자성별', '사고자학년', '사고발생일',
       '사고발생요일', '사고발생시각', '사고시간', '사고장소', '사고부위', '사고형태', '사고당시활동', '사고매개물',
       '연도', '사고월', '계절'],
      dtype='object')

In [8]:
# 사고자구분 중 '일반학생', '특수학교(학급)학생', '체육특기학생' 값만 추출
print('추출 전:', df.사고자구분.unique())

sch_df = df[df['사고자구분'].isin(['일반학생', '특수학교(학급)학생', '체육특기학생'])]
print('추출 후:', sch_df.사고자구분.unique())

# 연도 type str로 변경
sch_df['연도'] = sch_df['연도'].astype(str)

추출 전: ['일반학생' '특수학교(학급)학생' '체육특기학생' '교직원' '교육활동참여자' '교직원(계약직)' '교직원(일반직)' nan
 '교직원(교원)' '일반인' '기타(일반인 등)']
추출 후: ['일반학생' '특수학교(학급)학생' '체육특기학생']


C:\Users\cabby\AppData\Local\Temp\ipykernel_35172\750086091.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sch_df['연도'] = sch_df['연도'].astype(str)


##### 1-3-1. 학교급과 사고 수 간의 관계 분석

In [26]:
# sch_acci_corr 데이터프레임(5개년 누적, 초등학교 저학년/고학년 구분O)
sch_acci_corr = sch_df[~sch_df['학교급'].isin(['기타학교', '특수학교'])]
# 초등학교 저학년/고학년 구분
sch_acci_corr.loc[(sch_acci_corr['학교급']=='초등학교')&(sch_acci_corr['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_acci_corr.loc[(sch_acci_corr['학교급']=='초등학교')&(sch_acci_corr['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 연도, 학교급 기준으로 groupby
sch_acci_corr = sch_acci_corr.groupby(['학교급']).size().reset_index(name='총 사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교"]
sch_acci_corr['학교급'] = pd.Categorical(sch_acci_corr['학교급'], categories=custom_order, ordered=True)
sch_acci_corr = sch_acci_corr.sort_values(['학교급']).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_corr = sch_acci_corr.dropna()
sch_acci_corr 

,학교급,총 사고수
0,유치원,42054
1,초등학교(저학년),79010
2,초등학교(고학년),139056
3,중학교,215892
4,고등학교,129815


In [28]:
# Barplot 시각화
fig = px.bar(sch_acci_corr, x='학교급', y='총 사고수',
             category_orders={"학교급": ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교"]},
             title="학교급별 5개년 누적 총 사고수")

fig.update_layout(xaxis_title='학교급', yaxis_title='총 사고수')
fig.show()

In [ ]:
# sch_acci_corr2 데이터프레임(5개년 누적, 초등학교 저학년/고학년 구분O)
sch_acci_corr2 = sch_df[~sch_df['학교급'].isin(['기타학교', '특수학교'])]
# 초등학교 저학년/고학년 구분
sch_acci_corr2.loc[(sch_acci_corr2['학교급']=='초등학교')&(sch_acci_corr2['사고자학년'].isin(['1학년','2학년','3학년'])),'학교급']='초등학교(저학년)'
sch_acci_corr2.loc[(sch_acci_corr2['학교급']=='초등학교')&(sch_acci_corr2['사고자학년'].isin(['4학년','5학년','6학년'])),'학교급']='초등학교(고학년)'
# 연도, 학교급 기준으로 groupby
sch_acci_corr2 = sch_acci_corr2.groupby(['학교급']).size().reset_index(name='총 사고수')
# 연도, 학교급 기준으로 정렬
custom_order = ["유치원", "초등학교(저학년)", "초등학교(고학년)", "중학교", "고등학교"]
sch_acci_corr2['학교급'] = pd.Categorical(sch_acci_corr2['학교급'], categories=custom_order, ordered=True)
sch_acci_corr2 = sch_acci_corr2.sort_values(['학교급']).reset_index(drop=True)
# 초등학교 저학년/고학년 구분 과정에서 생긴 nan 값(사고자학년 값이 유아인 경우) 처리
sch_acci_corr2 = sch_acci_corr2.dropna()
sch_acci_corr2 